# Global Tweet Sentiment Analysis - Mood of the World { HTML Report }

# Install Required Packages

In [ ]:
!pip install tweepy textblob plotly pandas numpy wordcloud matplotlib seaborn python-dotenv scikit-learn --quiet
!python -m textblob.download_corpora

# Import Libraries

In [ ]:
import tweepy
import pandas as pd
import numpy as np
from textblob import TextBlob
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import re
import random
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pyo.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "colab"
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)



# Twitter API Configuration
## Note : get API from https://developer.twitter.com/


In [ ]:
class TwitterConfig:
    def __init__(self):
        # Replace with your actual API credentials
        self.API_KEY = "YOUR_API_KEY"
        self.API_SECRET = "YOUR_API_SECRET"
        self.ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
        self.ACCESS_TOKEN_SECRET = "YOUR_ACCESS_TOKEN_SECRET"
        self.BEARER_TOKEN = "YOUR_BEARER_TOKEN"

        # Rate limit settings
        self.FREE_TIER_MONTHLY_LIMIT = 100
        self.DAILY_BUDGET = 10
        self.BATCH_SIZE = 25
        self.REQUEST_DELAY = 1.5

    def get_api_v1(self):
        """Get Twitter API v1.1 client"""
        auth = tweepy.OAuthHandler(self.API_KEY, self.API_SECRET)
        auth.set_access_token(self.ACCESS_TOKEN, self.ACCESS_TOKEN_SECRET)
        return tweepy.API(auth, wait_on_rate_limit=True)

    def get_api_v2(self):
        """Get Twitter API v2 client"""
        return tweepy.Client(
            bearer_token=self.BEARER_TOKEN,
            consumer_key=self.API_KEY,
            consumer_secret=self.API_SECRET,
            access_token=self.ACCESS_TOKEN,
            access_token_secret=self.ACCESS_TOKEN_SECRET,
            wait_on_rate_limit=True
        )
config = TwitterConfig()

# Data Collection Via API or Sample Data for Demonstration

In [ ]:
class GlobalTweetCollector:
    def __init__(self, api_v1=None, api_v2=None):
        self.api_v1 = api_v1
        self.api_v2 = api_v2
        self.tweets_data = []

    def clean_tweet(self, text):
        """tweet cleaning"""
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'@\w+|#\w+', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]', '', text)
        return text.strip()

    def get_enhanced_sentiment(self, text):
        """sentiment analysis with confidence scores"""
        try:
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity
            if polarity > 0.2:
                sentiment = 'Positive'
                confidence = abs(polarity)
            elif polarity < -0.2:
                sentiment = 'Negative'
                confidence = abs(polarity)
            else:
                sentiment = 'Neutral'
                confidence = 1 - abs(polarity)
            emotion = self.detect_emotion(text.lower())

            return {
                'sentiment': sentiment,
                'polarity': polarity,
                'subjectivity': subjectivity,
                'confidence': confidence,
                'emotion': emotion,
                'word_count': len(text.split())
            }
        except Exception as e:
            return {
                'sentiment': 'Neutral',
                'polarity': 0.0,
                'subjectivity': 0.0,
                'confidence': 0.0,
                'emotion': 'neutral',
                'word_count': 0
            }

    def detect_emotion(self, text):
        """Detect basic emotions from text"""
        emotion_keywords = {
            'joy': ['happy', 'excited', 'great', 'wonderful', 'amazing', 'love', 'perfect'],
            'sadness': ['sad', 'depressed', 'down', 'upset', 'disappointed', 'lonely'],
            'anger': ['angry', 'mad', 'furious', 'annoyed', 'frustrated', 'hate'],
            'fear': ['scared', 'afraid', 'worried', 'anxious', 'nervous', 'panic'],
            'surprise': ['surprised', 'shocked', 'unexpected', 'wow', 'omg'],
            'gratitude': ['grateful', 'thankful', 'blessed', 'appreciate', 'lucky']
        }

        emotion_scores = {}
        for emotion, keywords in emotion_keywords.items():
            score = sum(1 for keyword in keywords if keyword in text)
            if score > 0:
                emotion_scores[emotion] = score

        if emotion_scores:
            return max(emotion_scores, key=emotion_scores.get)
        return 'neutral'

    def collect_tweets_with_api(self, keywords=None, count=50):
        """Collect tweets using Twitter API"""
        if not self.api_v2:
            print("Twitter API not configured")
            return pd.DataFrame()

        if keywords is None:
            keywords = ["feeling good", "having a great day", "feeling down", "stressed out", "grateful today"]

        print(f"Collecting tweets using Twitter API...")
        print(f"Target: {count} tweets across {len(keywords)} keywords")

        tweets_per_keyword = max(1, count // len(keywords))

        for i, keyword in enumerate(keywords):
            if len(self.tweets_data) >= count:
                break

            try:
                print(f"Searching: '{keyword}' ({i+1}/{len(keywords)})")

                tweets = self.api_v2.search_recent_tweets(
                    query=f'"{keyword}" -is:retweet lang:en',
                    tweet_fields=['created_at', 'author_id', 'public_metrics', 'context_annotations'],
                    max_results=min(tweets_per_keyword, 100)
                )

                if tweets.data:
                    for tweet in tweets.data:
                        if len(self.tweets_data) >= count:
                            break

                        cleaned_text = self.clean_tweet(tweet.text)
                        if len(cleaned_text) > 15:
                            sentiment_data = self.get_enhanced_sentiment(cleaned_text)

                            tweet_data = {
                                'id': tweet.id,
                                'text': cleaned_text,
                                'original_text': tweet.text,
                                'created_at': tweet.created_at,
                                'author_id': tweet.author_id,
                                'keyword': keyword,
                                'source': 'twitter_api',
                                **sentiment_data,
                                'retweet_count': tweet.public_metrics.get('retweet_count', 0) if tweet.public_metrics else 0,
                                'like_count': tweet.public_metrics.get('like_count', 0) if tweet.public_metrics else 0,
                            }

                            self.tweets_data.append(tweet_data)

                import time
                time.sleep(config.REQUEST_DELAY)

            except Exception as e:
                print(f"Error with '{keyword}': {str(e)}")
                if "rate limit" in str(e).lower():
                    print("Rate limit reached - switching to sample data")
                    break

        print(f"Collected {len(self.tweets_data)} real tweets")
        return pd.DataFrame(self.tweets_data)

    def generate_realistic_sample_data(self, size=1000):
        """Generate realistic sample data with patterns"""
        print(f" Generating {size} realistic sample tweets...")

        tweet_templates = {
            'positive': [
                "Just had the most amazing coffee this morning! ☀️",
                "Feeling incredibly grateful for my family today 💕",
                "Beautiful sunset tonight, life is good! 🌅",
                "Finally finished that project I've been working on! 🎉",
                "Met up with old friends today, such a perfect day! 😊",
                "Got some great news today, feeling so blessed! ✨",
                "Nothing beats a good workout to start the day! 💪",
                "Weekend vibes are hitting different today! 🌟",
                "Sometimes the little things make me the happiest 💝",
                "Accomplished so much today, feeling proud! 🏆"
            ],
            'negative': [
                "Having one of those days where nothing goes right 😔",
                "Really struggling to find motivation lately",
                "Work has been incredibly stressful this week",
                "Feeling overwhelmed with everything going on",
                "Just need a break from all this chaos",
                "When will things start looking up? 😞",
                "Another sleepless night ahead of me",
                "Can't seem to catch a break these days",
                "Feeling disconnected from everyone lately",
                "This weather is really getting me down"
            ],
            'neutral': [
                "Just another typical Monday morning",
                "Regular day at the office, nothing special",
                "Weather looks okay for the weekend",
                "Going through my usual routine today",
                "Standard Tuesday, same as always",
                "Just checking in, hope everyone's well",
                "Another day, another dollar as they say",
                "Normal evening at home tonight",
                "Regular coffee meeting this afternoon",
                "Just going with the flow today"
            ]
        }

        emotions_dist = {'positive': 0.45, 'negative': 0.25, 'neutral': 0.30}
        sample_data = []
        end_date = datetime.now()
        start_date = end_date - timedelta(days=7)
        timestamps = pd.date_range(start=start_date, end=end_date, periods=size)

        for i, timestamp in enumerate(timestamps):
            hour = timestamp.hour
            day_of_week = timestamp.weekday()

            if 7 <= hour <= 10:
                pos_boost = 0.15
            elif 18 <= hour <= 21:
                pos_boost = 0.10
            elif 23 <= hour <= 2:
                pos_boost = -0.20
            elif day_of_week >= 5:
                pos_boost = 0.12
            else:
                pos_boost = 0

            adjusted_pos = min(0.7, emotions_dist['positive'] + pos_boost)
            adjusted_neg = max(0.1, emotions_dist['negative'] - pos_boost/2)
            adjusted_neu = 1 - adjusted_pos - adjusted_neg

            sentiment_choice = np.random.choice(
                ['positive', 'negative', 'neutral'],
                p=[adjusted_pos, adjusted_neg, adjusted_neu]
            )

            template = np.random.choice(tweet_templates[sentiment_choice])

            variations = {
                'positive': ['!', ' 🙂', ' Today was good.', ' Feeling blessed.'],
                'negative': ['...', ' 😕', ' Not my best day.', ' Hope tomorrow is better.'],
                'neutral': ['.', ' Just saying.', ' That\'s life.', ' Moving on.']
            }

            text = template + np.random.choice(variations[sentiment_choice])

            sentiment_data = self.get_enhanced_sentiment(text)

            if sentiment_choice == 'positive':
                likes = np.random.poisson(8)
                retweets = np.random.poisson(2)
            elif sentiment_choice == 'negative':
                likes = np.random.poisson(3)
                retweets = np.random.poisson(1)
            else:
                likes = np.random.poisson(4)
                retweets = np.random.poisson(1)

            sample_data.append({
                'id': f'sample_{i}',
                'text': text,
                'original_text': text,
                'created_at': timestamp,
                'author_id': f'user_{i % 100}',
                'keyword': np.random.choice(['mood', 'feeling', 'day', 'life', 'today']),
                'source': 'sample_data',
                **sentiment_data,
                'like_count': likes,
                'retweet_count': retweets,
            })

        print(f"Generated {len(sample_data)} realistic sample tweets")
        return pd.DataFrame(sample_data)

# Initialize APIs and Collect Data

In [ ]:
def initialize_data_collection():
    """Initialize data collection with fallback options"""
    collector = GlobalTweetCollector()
    df_final = pd.DataFrame()

    try:
        print("Attempting Twitter API connection...")
        api_v1 = config.get_api_v1()
        api_v2 = config.get_api_v2()

        me = api_v1.verify_credentials()
        if me:
            print(f"Connected as: @{me.screen_name}")
            collector.api_v1 = api_v1
            collector.api_v2 = api_v2

            df_real = collector.collect_tweets_with_api(count=config.DAILY_BUDGET)
            if len(df_real) > 0:
                df_final = pd.concat([df_final, df_real], ignore_index=True)
                print(f"Added {len(df_real)} real tweets")

    except Exception as e:
        print(f"Twitter API issue: {str(e)[:100]}...")
        print("Continuing with sample data only")

    print("\nGenerating comprehensive sample dataset...")
    df_sample = collector.generate_realistic_sample_data(1200)
    df_final = pd.concat([df_final, df_sample], ignore_index=True)

    df_final['created_at'] = pd.to_datetime(df_final['created_at'])
    df_final['hour'] = df_final['created_at'].dt.hour
    df_final['day_of_week'] = df_final['created_at'].dt.day_name()
    df_final['date'] = df_final['created_at'].dt.date
    df_final['is_weekend'] = df_final['created_at'].dt.weekday >= 5

    print(f"\n Final dataset ready!")
    print(f"Total tweets: {len(df_final):,}")
    print(f"Real tweets: {len(df_final[df_final['source'] == 'twitter_api']):,}")
    print(f"Sample tweets: {len(df_final[df_final['source'] == 'sample_data']):,}")
    print(f"Date range: {df_final['created_at'].min().date()} to {df_final['created_at'].max().date()}")

    return df_final, collector

df_tweets, collector = initialize_data_collection()

# Data Analysis

In [ ]:
class EnhancedSentimentAnalyzer:
    def __init__(self, df):
        self.df = df.copy()
        self.summary_stats = self.calculate_summary_stats()

    def calculate_summary_stats(self):
        """Calculate comprehensive summary statistics"""
        total_tweets = len(self.df)
        sentiment_counts = self.df['sentiment'].value_counts()

        return {
            'total_tweets': total_tweets,
            'unique_users': self.df['author_id'].nunique(),
            'date_range': (self.df['created_at'].min(), self.df['created_at'].max()),
            'positive_pct': (sentiment_counts.get('Positive', 0) / total_tweets) * 100,
            'negative_pct': (sentiment_counts.get('Negative', 0) / total_tweets) * 100,
            'neutral_pct': (sentiment_counts.get('Neutral', 0) / total_tweets) * 100,
            'avg_polarity': self.df['polarity'].mean(),
            'avg_subjectivity': self.df['subjectivity'].mean(),
            'avg_confidence': self.df['confidence'].mean(),
            'avg_word_count': self.df['word_count'].mean(),
            'total_engagement': self.df['like_count'].sum() + self.df['retweet_count'].sum(),
            'top_emotions': self.df['emotion'].value_counts().head().to_dict()
        }

    def get_time_analysis(self):
        """Comprehensive time-based analysis"""
        hourly = self.df.groupby('hour').agg({
            'polarity': ['mean', 'std', 'count'],
            'subjectivity': 'mean',
            'confidence': 'mean',
            'like_count': 'sum'
        }).round(3)

        daily = self.df.groupby('day_of_week').agg({
            'polarity': ['mean', 'std', 'count'],
            'subjectivity': 'mean',
            'confidence': 'mean'
        }).round(3)

        return hourly, daily

    def get_emotion_analysis(self):
        """Analyze emotional patterns"""
        emotion_sentiment = pd.crosstab(self.df['emotion'], self.df['sentiment'], normalize='index') * 100
        emotion_by_hour = self.df.groupby(['hour', 'emotion']).size().unstack(fill_value=0)

        return emotion_sentiment, emotion_by_hour

    def get_engagement_analysis(self):
        """Analyze engagement patterns"""
        engagement_by_sentiment = self.df.groupby('sentiment').agg({
            'like_count': ['mean', 'sum'],
            'retweet_count': ['mean', 'sum'],
            'confidence': 'mean'
        }).round(2)

        return engagement_by_sentiment

analyzer = EnhancedSentimentAnalyzer(df_tweets)
summary = analyzer.summary_stats

print("\n MOOD of the World Analysis")
print("=" * 50)
print(f"Dataset Overview:")
print(f"   • Total Tweets: {summary['total_tweets']:,}")
print(f"   • Unique Users: {summary['unique_users']:,}")
print(f"   • Date Range: {summary['date_range'][0].date()} to {summary['date_range'][1].date()}")
print(f"\n Sentiment Distribution:")
print(f"   • Positive: {summary['positive_pct']:.1f}%")
print(f"   • Negative: {summary['negative_pct']:.1f}%")
print(f"   • Neutral: {summary['neutral_pct']:.1f}%")
print(f"\n Quality Metrics:")
print(f"   • Average Polarity: {summary['avg_polarity']:.3f}")
print(f"   • Average Confidence: {summary['avg_confidence']:.3f}")
print(f"   • Average Word Count: {summary['avg_word_count']:.1f}")
print(f"\n Engagement:")
print(f"   • Total Likes + Retweets: {summary['total_engagement']:,}")
print(f"\n Top Emotions:")
for emotion, count in summary['top_emotions'].items():
    print(f"   • {emotion.title()}: {count:,}")

# Standard Plotting Functions and Time Series

In [ ]:
class StandardPlotter:
    def __init__(self, df, analyzer):
        self.df = df
        self.analyzer = analyzer
        # plt.style.use('default')  # Changed from 'seaborn-v0_8' which might not be available
        plt.rcParams['figure.facecolor'] = 'white'
        plt.rcParams['axes.facecolor'] = 'white'

    def plot_sentiment_overview(self, figsize=(15, 10)):
        """Create comprehensive standard plots overview"""
        fig, axes = plt.subplots(2, 3, figsize=figsize)
        fig.suptitle('Sentiment Analysis - Standard Plots', fontsize=16, fontweight='bold')

        sentiment_counts = self.df['sentiment'].value_counts()
        colors = ['#2E86AB', '#A23B72', '#F18F01']
        wedges, texts, autotexts = axes[0, 0].pie(
            sentiment_counts.values,
            labels=sentiment_counts.index,
            autopct='%1.1f%%',
            colors=colors,
            startangle=90,
            textprops={'fontsize': 10}
        )
        axes[0, 0].set_title('Sentiment Distribution', fontweight='bold')

        axes[0, 1].hist(self.df['polarity'], bins=30, color='#2E86AB', alpha=0.7, edgecolor='black')
        mean_polarity = self.df['polarity'].mean()
        axes[0, 1].axvline(mean_polarity, color='red', linestyle='--', linewidth=2,
                          label=f'Mean: {mean_polarity:.3f}')
        axes[0, 1].set_title('Polarity Distribution', fontweight='bold')
        axes[0, 1].set_xlabel('Polarity Score')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        hourly_sentiment = self.df.groupby(['hour', 'sentiment']).size().unstack(fill_value=0)
        hourly_sentiment.plot(kind='bar', ax=axes[0, 2], color=colors, width=0.8)
        axes[0, 2].set_title('Sentiment by Hour of Day', fontweight='bold')
        axes[0, 2].set_xlabel('Hour')
        axes[0, 2].set_ylabel('Tweet Count')
        axes[0, 2].legend(title='Sentiment', loc='upper right')
        axes[0, 2].tick_params(axis='x', rotation=45)
        axes[0, 2].grid(True, alpha=0.3)

        scatter = axes[1, 0].scatter(
            self.df['subjectivity'],
            self.df['polarity'],
            c=self.df['confidence'],
            cmap='viridis',
            alpha=0.6,
            s=30
        )
        axes[1, 0].set_title('Polarity vs Subjectivity', fontweight='bold')
        axes[1, 0].set_xlabel('Subjectivity')
        axes[1, 0].set_ylabel('Polarity')
        axes[1, 0].grid(True, alpha=0.3)
        cbar = plt.colorbar(scatter, ax=axes[1, 0])
        cbar.set_label('Confidence', rotation=270, labelpad=15)

        emotion_counts = self.df['emotion'].value_counts().head(8)
        bars = axes[1, 1].bar(
            range(len(emotion_counts)),
            emotion_counts.values,
            color=plt.cm.Set3(np.linspace(0, 1, len(emotion_counts)))
        )
        axes[1, 1].set_title('Top Emotions', fontweight='bold')
        axes[1, 1].set_xlabel('Emotion')
        axes[1, 1].set_ylabel('Count')
        axes[1, 1].set_xticks(range(len(emotion_counts)))
        axes[1, 1].set_xticklabels(emotion_counts.index, rotation=45, ha='right')
        axes[1, 1].grid(True, alpha=0.3)

        for bar, value in zip(bars, emotion_counts.values):
            axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(emotion_counts.values)*0.01,
                           str(value), ha='center', va='bottom', fontweight='bold')

        daily_polarity = self.df.groupby(self.df['created_at'].dt.date)['polarity'].mean()
        axes[1, 2].plot(
            daily_polarity.index,
            daily_polarity.values,
            marker='o',
            linewidth=2,
            markersize=4,
            color='#2E86AB'
        )
        axes[1, 2].set_title('Daily Sentiment Trend', fontweight='bold')
        axes[1, 2].set_xlabel('Date')
        axes[1, 2].set_ylabel('Average Polarity')
        axes[1, 2].tick_params(axis='x', rotation=45)
        axes[1, 2].grid(True, alpha=0.3)

        if len(daily_polarity) > 1:
            z = np.polyfit(range(len(daily_polarity)), daily_polarity.values, 1)
            p = np.poly1d(z)
            axes[1, 2].plot(daily_polarity.index, p(range(len(daily_polarity))),
                           "r--", alpha=0.8, linewidth=1, label='Trend')
            axes[1, 2].legend()

        plt.tight_layout()
        plt.subplots_adjust(top=0.93)
        plt.show()
        plt.draw()

        return fig

    def plot_time_analysis(self, figsize=(14, 10)):
        """Time-based analysis plots"""
        fig, axes = plt.subplots(2, 2, figsize=figsize)
        fig.suptitle('Time-Based Sentiment Analysis', fontsize=16, fontweight='bold')

        hourly_sentiment = self.df.groupby(['hour', 'sentiment']).size().unstack(fill_value=0)
        hourly_pct = hourly_sentiment.div(hourly_sentiment.sum(axis=1), axis=0) * 100

        im = axes[0, 0].imshow(hourly_pct.T.values, cmap='RdYlBu_r', aspect='auto', interpolation='nearest')
        axes[0, 0].set_title('Hourly Sentiment Distribution (%)', fontweight='bold')
        axes[0, 0].set_xlabel('Hour of Day')
        axes[0, 0].set_ylabel('Sentiment')
        axes[0, 0].set_xticks(range(0, 24, 2))
        axes[0, 0].set_xticklabels(range(0, 24, 2))
        axes[0, 0].set_yticks(range(len(hourly_pct.columns)))
        axes[0, 0].set_yticklabels(hourly_pct.columns)

        cbar = plt.colorbar(im, ax=axes[0, 0])
        cbar.set_label('Percentage', rotation=270, labelpad=15)

        for i in range(len(hourly_pct.columns)):
            for j in range(len(hourly_pct.index)):
                if hourly_pct.T.iloc[i, j] > 0:
                    text = axes[0, 0].text(j, i, f'{hourly_pct.T.iloc[i, j]:.0f}%',
                                         ha="center", va="center", color="black", fontsize=8)

        weekend_sentiment = self.df.groupby(['is_weekend', 'sentiment']).size().unstack(fill_value=0)
        weekend_sentiment.plot(kind='bar', ax=axes[0, 1],
                             color=['#2E86AB', '#A23B72', '#F18F01'],
                             width=0.7)
        axes[0, 1].set_title('Weekend vs Weekday Sentiment', fontweight='bold')
        axes[0, 1].set_xlabel('Day Type')
        axes[0, 1].set_xticklabels(['Weekday', 'Weekend'], rotation=0)
        axes[0, 1].legend(title='Sentiment', loc='upper right')
        axes[0, 1].grid(True, alpha=0.3)

        for container in axes[0, 1].containers:
            axes[0, 1].bar_label(container, label_type='edge', fontsize=9)

        hourly_activity = self.df.groupby('hour').size()
        bars = axes[1, 0].bar(hourly_activity.index, hourly_activity.values,
                             color='#2E86AB', alpha=0.7, edgecolor='black')
        axes[1, 0].set_title('Tweet Activity by Hour', fontweight='bold')
        axes[1, 0].set_xlabel('Hour of Day')
        axes[1, 0].set_ylabel('Tweet Count')
        axes[1, 0].grid(axis='y', alpha=0.3)

        peak_hour = hourly_activity.idxmax()
        axes[1, 0].bar(peak_hour, hourly_activity[peak_hour],
                      color='#F18F01', alpha=0.9, edgecolor='black')
        axes[1, 0].text(peak_hour, hourly_activity[peak_hour] + max(hourly_activity)*0.02,
                       'Peak', ha='center', va='bottom', fontweight='bold')

        daily_emotions = self.df.groupby([self.df['created_at'].dt.date, 'emotion']).size().unstack(fill_value=0)
        top_emotions = self.df['emotion'].value_counts().head(5).index

        colors_emotion = plt.cm.Set1(np.linspace(0, 1, len(top_emotions)))

        for i, emotion in enumerate(top_emotions):
            if emotion in daily_emotions.columns:
                axes[1, 1].plot(daily_emotions.index, daily_emotions[emotion],
                               marker='o', label=emotion.title(), linewidth=2,
                               markersize=4, color=colors_emotion[i])

        axes[1, 1].set_title('Daily Emotion Trends', fontweight='bold')
        axes[1, 1].set_xlabel('Date')
        axes[1, 1].set_ylabel('Count')
        axes[1, 1].legend(loc='upper right')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.subplots_adjust(top=0.93)

        plt.show()
        plt.draw()

        return fig

standard_plotter = StandardPlotter(df_tweets, analyzer)

print("Standard Visualizations...")
print("=" * 50)

print("\n Sentiment Overview... \n")
overview_fig = standard_plotter.plot_sentiment_overview()

print("\n Time Analysis...\n")
time_fig = standard_plotter.plot_time_analysis()

print(f"\n Quick Stats:\n")
print(f"   • Most active hour: {df_tweets.groupby('hour').size().idxmax()}:00")
print(f"   • Most positive hour: {df_tweets.groupby('hour')['polarity'].mean().idxmax()}:00")
print(f"   • Weekend mood: {df_tweets[df_tweets['is_weekend']]['polarity'].mean():.3f}")
print(f"   • Weekday mood: {df_tweets[~df_tweets['is_weekend']]['polarity'].mean():.3f}")
print(f"   • Top emotion: {df_tweets['emotion'].mode().iloc[0].title()}")

# in this version the application uses free API with rate limit to retrive the data also a sample data of 1200 is used for demonstration. So the program runs without Online API also

## Expectation on upcoming version continuation
* More Detailed and interactive Graph
* Advanced Analytics and Insights
* And Many more in future

# Html Report

In [ ]:
def generate_sample_data(n_samples=1000):
    emotions = ['joy', 'sadness', 'anger', 'fear', 'surprise', 'disgust', 'trust', 'anticipation']
    sentiments = ['Positive', 'Negative', 'Neutral']
    data = []
    base_date = datetime.now() - timedelta(days=7)

    for i in range(n_samples):
        random_date = base_date + timedelta(
            days=random.randint(0, 6),
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59)
        )

        polarity = random.uniform(-1, 1)
        subjectivity = random.uniform(0, 1)
        confidence = random.uniform(0.5, 1.0)
        if polarity > 0.1:
            sentiment = 'Positive'
        elif polarity < -0.1:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'

        sample_texts = [
            "Having a great day today! #happy #blessed",
            "Feeling frustrated with the current situation",
            "Just another ordinary day at work",
            "Excited about the upcoming weekend plans",
            "Weather is perfect for a walk in the park",
            "Technology is amazing these days",
            "Missing the good old times",
            "Looking forward to new opportunities"
        ]

        data.append({
            'text': random.choice(sample_texts),
            'created_at': random_date,
            'hour': random_date.hour,
            'sentiment': sentiment,
            'emotion': random.choice(emotions),
            'polarity': polarity,
            'subjectivity': subjectivity,
            'confidence': confidence,
            'like_count': random.randint(0, 100),
            'retweet_count': random.randint(0, 50)
        })

    return pd.DataFrame(data)

print("Generating data...")
df_tweets = generate_sample_data(1000)
print(f"Generated {len(df_tweets)} tweets")

class InteractivePlotter:
    def __init__(self, df):
        self.df = df

    def create_sentiment_dashboard(self):
        """Interactive Dashboard"""
        print("Sentiment Dashboard...")

        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=('Sentiment Distribution', 'Hourly Sentiment Patterns',
                          'Emotion Analysis', 'Engagement vs Sentiment',
                          'Polarity vs Subjectivity', 'Daily Trend'),
            specs=[[{"type": "pie"}, {"type": "bar"}],
                   [{"type": "bar"}, {"type": "scatter"}],
                   [{"type": "scatter"}, {"type": "scatter"}]]
        )
        sentiment_counts = self.df['sentiment'].value_counts()
        fig.add_trace(go.Pie(
            labels=sentiment_counts.index,
            values=sentiment_counts.values,
            marker_colors=['#2E86AB', '#A23B72', '#F18F01'],
            hole=0.3,
            name="Sentiment"
        ), row=1, col=1)

        hourly_sentiment = self.df.groupby(['hour', 'sentiment']).size().unstack(fill_value=0)
        colors_sentiment = {'Positive': '#2E86AB', 'Negative': '#A23B72', 'Neutral': '#F18F01'}
        for sentiment in hourly_sentiment.columns:
            color = colors_sentiment.get(sentiment, '#666666')
            fig.add_trace(go.Bar(
                x=hourly_sentiment.index,
                y=hourly_sentiment[sentiment],
                name=f'Hourly {sentiment}',
                marker_color=color,
                showlegend=False
            ), row=1, col=2)

        emotion_counts = self.df['emotion'].value_counts().head(8)
        fig.add_trace(go.Bar(
            x=emotion_counts.index,
            y=emotion_counts.values,
            marker_color=px.colors.qualitative.Set3[:len(emotion_counts)],
            name="Emotions",
            showlegend=False
        ), row=2, col=1)

        engagement_sentiment = self.df.groupby('sentiment').agg({
            'like_count': 'mean',
            'retweet_count': 'mean'
        }).reset_index()

        fig.add_trace(go.Scatter(
            x=engagement_sentiment['like_count'],
            y=engagement_sentiment['retweet_count'],
            mode='markers+text',
            text=engagement_sentiment['sentiment'],
            textposition='top center',
            marker=dict(
                size=20,
                color=['#2E86AB', '#A23B72', '#F18F01'][:len(engagement_sentiment)]
            ),
            name="Engagement",
            showlegend=False
        ), row=2, col=2)

        fig.add_trace(go.Scatter(
            x=self.df['subjectivity'],
            y=self.df['polarity'],
            mode='markers',
            marker=dict(
                color=self.df['confidence'],
                colorscale='viridis',
                size=4,
                opacity=0.6,
                showscale=True,
                colorbar=dict(
                    title="Confidence",
                    x=0.45,
                    len=0.3
                )
            ),
            name="Polarity vs Subjectivity",
            showlegend=False
        ), row=3, col=1)

        daily_polarity = self.df.groupby(self.df['created_at'].dt.date)['polarity'].mean()
        fig.add_trace(go.Scatter(
            x=daily_polarity.index,
            y=daily_polarity.values,
            mode='lines+markers',
            line=dict(color='#2E86AB', width=3),
            marker=dict(size=8),
            name="Daily Trend",
            showlegend=False
        ), row=3, col=2)

        fig.update_layout(
            height=1200,
            title_text="Mood of the World Analysis Dashboard",
            title_font_size=20,
            title_x=0.5,
            showlegend=False
        )

        return fig

    def create_emotion_heatmap(self):
        """emotion-time heatmap"""
        print("Emotion Heatmap...")

        emotion_hour = self.df.groupby(['hour', 'emotion']).size().unstack(fill_value=0)

        top_emotions = self.df['emotion'].value_counts().head(8).index
        emotion_hour_filtered = emotion_hour[top_emotions]

        fig = go.Figure(data=go.Heatmap(
            z=emotion_hour_filtered.T.values,
            x=emotion_hour_filtered.index,
            y=emotion_hour_filtered.columns,
            colorscale='viridis',
            hoverongaps=False,
            hovertemplate='Hour: %{x}<br>Emotion: %{y}<br>Count: %{z}<extra></extra>'
        ))

        fig.update_layout(
            title='Emotion Patterns Throughout the Day',
            title_font_size=16,
            title_x=0.5,
            xaxis_title='Hour of Day',
            yaxis_title='Emotion',
            height=600,
            width=800
        )

        return fig

    def create_word_frequency_chart(self):
        """Create word frequency visualization"""
        print("word frequency chart...")

        all_text = ' '.join(self.df['text'].astype(str))

        words = re.findall(r'\b[a-zA-Z]{3,}\b', all_text.lower())
        stop_words = {
            'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by',
            'i', 'you', 'he', 'she', 'it', 'we', 'they', 'this', 'that', 'is', 'are', 'was', 'were',
            'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should',
            'may', 'might', 'can', 'cant', 'dont', 'wont', 'im', 'youre', 'hes', 'shes', 'its', 'were',
            'theyre', 'today', 'just', 'get', 'got', 'going', 'go', 'like', 'really', 'know', 'time',
            'day', 'way', 'think', 'feel', 'feeling', 'good', 'great', 'bad', 'new', 'one', 'two'
        }

        filtered_words = [word for word in words if word not in stop_words and len(word) > 3]
        word_freq = Counter(filtered_words).most_common(25)

        if word_freq:
            words_list, counts_list = zip(*word_freq)
        else:
            words_list, counts_list = ['sample', 'words', 'example'], [10, 8, 5]

        fig = go.Figure(data=go.Bar(
            x=list(words_list),
            y=list(counts_list),
            marker_color=px.colors.sequential.Viridis,
            hovertemplate='Word: %{x}<br>Frequency: %{y}<extra></extra>'
        ))

        fig.update_layout(
            title='Most Frequent Words in Tweets',
            title_font_size=16,
            title_x=0.5,
            xaxis_title='Words',
            yaxis_title='Frequency',
            height=500,
            width=800,
            xaxis_tickangle=-45
        )

        return fig

    def create_time_series_analysis(self):
        """Create detailed time series analysis"""
        print("time series analysis...")

        # Hourly sentiment trends
        hourly_data = self.df.groupby('hour').agg({
            'polarity': 'mean',
            'subjectivity': 'mean',
            'confidence': 'mean'
        }).reset_index()

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=hourly_data['hour'],
            y=hourly_data['polarity'],
            mode='lines+markers',
            name='Polarity',
            line=dict(color='#2E86AB', width=3),
            marker=dict(size=8)
        ))

        fig.add_trace(go.Scatter(
            x=hourly_data['hour'],
            y=hourly_data['subjectivity'],
            mode='lines+markers',
            name='Subjectivity',
            line=dict(color='#A23B72', width=3),
            marker=dict(size=8),
            yaxis='y2'
        ))

        fig.update_layout(
            title='Hourly Sentiment Patterns',
            title_font_size=16,
            title_x=0.5,
            xaxis_title='Hour of Day',
            yaxis_title='Polarity Score',
            yaxis2=dict(
                title='Subjectivity Score',
                overlaying='y',
                side='right'
            ),
            height=500,
            width=800,
            hovermode='x unified'
        )

        return fig

interactive_plotter = InteractivePlotter(df_tweets)

print("Interactive Visualizations...")
print("=" * 50)

# Display data info
print(f"\n Dataset Info:")
print(f"Total tweets: {len(df_tweets)}")
print(f"Date range: {df_tweets['created_at'].min()} to {df_tweets['created_at'].max()}")
print(f"Sentiment distribution:\n{df_tweets['sentiment'].value_counts()}")

print("\n1.Sentiment Dashboard...")
dashboard_fig = interactive_plotter.create_sentiment_dashboard()
dashboard_fig.show()

print("\n2. Emotion Heatmap...")
emotion_heatmap = interactive_plotter.create_emotion_heatmap()
emotion_heatmap.show()

print("\n3. Word Frequency Analysis...")
word_freq_fig = interactive_plotter.create_word_frequency_chart()
word_freq_fig.show()

print("\n4. Time Series Analysis...")
time_series_fig = interactive_plotter.create_time_series_analysis()
time_series_fig.show()
dashboard_fig.write_html("sentiment_dashboard.html")
emotion_heatmap.write_html("emotion_heatmap.html")
word_freq_fig.write_html("word_frequency.html")
time_series_fig.write_html("time_series.html")
print("HTML files saved successfully!")